In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import os

project_path = '/content/drive/MyDrive/stock_price'
os.chdir(project_path)

# Verify
!ls

alpha_earnings_calls.csv       alpha_ticker_news.csv  merged_news_stock.csv
alpha_economic_indicators.csv  alpha_topic_news.csv   __pycache__
alpha_stock_prices.csv	       data_augmentation.py


In [2]:
!pip install transformers pandas torch

In [3]:
# install augmentation library dependencies
!pip install nlpaug

Combine stock_price file and tricker_news file.

In [10]:
import pandas as pd

# Load stock price data
stock_df = pd.read_csv("alpha_stock_prices.csv")

# Convert stock price 'date' to datetime format
stock_df['date'] = pd.to_datetime(stock_df['date'], format='mixed')
stock_df['date'] = stock_df['date'].dt.strftime("%Y-%m-%d")  # Normalize format

# Load news data
news_df = pd.read_csv("alpha_ticker_news.csv")

# Convert news 'date' to datetime format
news_df['date'] = pd.to_datetime(news_df['date'].str[:8], format="%Y%m%d")
news_df['date'] = news_df['date'].dt.strftime("%Y-%m-%d")  # Normalize format

# Merge the two DataFrames on 'ticker' and 'date'
merged_df = pd.merge(news_df, stock_df, on=["ticker", "date"], how="inner")


# Save result
merged_df.to_csv("merged_news_stock.csv", index=False)
print("Save merged_news_stock.csv")
merged_df.head()

Save merged_news_stock.csv


,ticker,title,summary,sentiment,score,date,open,high,low,close,volume
0,AAPL,Mag 7 Earnings Preview: What Can Investors Exp...,We get into the heart of the Q1 earnings seaso...,Neutral,0.075539,2025-04-25,206.365,209.75,206.2,209.28,38222258
1,AAPL,Should You Invest in Bitcoin Now?,Bitcoin has broken away from the NASDAQ and is...,Somewhat-Bullish,0.216740,2025-04-25,206.365,209.75,206.2,209.28,38222258
2,AAPL,Apple Plans iPhone Shift From China To India A...,Apple plans to import most U.S. iPhones from I...,Neutral,0.124279,2025-04-25,206.365,209.75,206.2,209.28,38222258
3,AAPL,Earnings Data Deluge,Pre-market futures are in the red slightly thi...,Neutral,0.047797,2025-04-25,206.365,209.75,206.2,209.28,38222258
4,AAPL,Comparing Apple With Industry Competitors In T...,In today's rapidly changing and highly competi...,Somewhat-Bullish,0.260158,2025-04-25,206.365,209.75,206.2,209.28,38222258


Disply stock price line chart

In [11]:
import plotly.express as px

# Create the line chart
fig = px.line(merged_df, x='date', y='close', color='ticker', title='Stock Prices Over Time')

# Customize the chart (optional)
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Closing Price',
    legend_title='Company Ticker'
)

# Show the chart
fig.show()

Remove outliers value , display stock prick line chart.

In [12]:
# Remove data for ticker "BRK-A"
merged_df = merged_df[merged_df["ticker"] != "BRK-A"]

# Save result
merged_df.to_csv("merged_news_stock.csv", index=False)
print("Save merged_news_stock.csv")
merged_df.head()


# Create the line chart
fig = px.line(merged_df, x='date', y='close', color='ticker', title='Stock Prices Over Time')

# Customize the chart (optional)
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Closing Price',
    legend_title='Company Ticker'
)

# Show the chart
fig.show()

Save merged_news_stock.csv


add finbert sentiment score to the file

In [13]:
from transformers import pipeline

# Load the FinBERT model and tokenizer
finbert = pipeline("sentiment-analysis", model="ProsusAI/finbert")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.

Device set to use cuda:0


In [14]:
import pandas as pd
merged_df = pd.read_csv("merged_news_stock.csv")

In [15]:
!pip install datasets
from tqdm import tqdm
from datasets import Dataset

Add data augmentation

In [20]:
from data_augmentation import FinancialTextAugmenter
import nltk
nltk.download('averaged_perceptron_tagger_eng')

augmenter_synonym = FinancialTextAugmenter(use_synonym=True, use_gpu=True)
augmenter_entity_mask = FinancialTextAugmenter(use_entity_mask=True, use_gpu=True)
augmenter_reorder = FinancialTextAugmenter(use_reorder=True, use_gpu=True)
augmenter_all = FinancialTextAugmenter(use_synonym=True, use_entity_mask=True, use_reorder=True, use_gpu=True)

def augment_df(df, augmenter, percentage=0.3):
    df = df.copy()
    augmented_rows = []
    n_rows = int(len(df) * percentage)

    df_sampled = df.sample(n=n_rows, random_state=42)

    print("Displaying first 10 augmented examples: ")
    i = 0
    for idx, row in df_sampled.iterrows():
        original_text = row['summary']
        augmented_text = augmenter.augment(original_text)
        if i < 10:
            print("Original: ", original_text)
            print("Augmented: ", augmented_text, "\n")
            i += 1

        if augmented_text and augmented_text != original_text:
            new_row = row.copy()
            new_row['summary'] = augmented_text
            augmented_rows.append(new_row)

    augmented_df = pd.DataFrame(augmented_rows)
    print(f"Number of augmented rows added to dataset: {len(augmented_df)}")
    combined_df = pd.concat([df, augmented_df], ignore_index=True)
    return combined_df


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [21]:
# Create a smaller df to test (your code)
df = merged_df

print("*****Augmenting data with SYNONYMS*****")
df_synonymed = augment_df(df, augmenter=augmenter_synonym, percentage=0.3)
print("*****Augmenting data with ENTITY MASK*****")
df_masked = augment_df(df, augmenter=augmenter_entity_mask, percentage=0.3)
print("*****Augmenting data with SENTENCE REORDERING*****")
df_reordered = augment_df(df, augmenter=augmenter_reorder, percentage=0.5)
print("*****Augmenting data with SYNONYMS->ENTITY MASK->SENTENCE REORDERING*****")
df_all_augmented = augment_df(df, augmenter=augmenter_all, percentage=0.3)

# Convert your pandas DataFrame to a Huggingface Dataset
dataset_synonymed = Dataset.from_pandas(df_synonymed)
dataset_masked = Dataset.from_pandas(df_masked)
dataset_reordered = Dataset.from_pandas(df_reordered)
dataset_all_augmented = Dataset.from_pandas(df_all_augmented)

# Define a function for batch sentiment scoring
def get_finbert_scores_batch(batch):
    try:
        results = finbert(batch['summary'])  # Run FinBERT on the batch
        scores = []
        for result in results:
            if result['label'] == 'positive':
                scores.append(result['score'])
            else:
                scores.append(-result['score'])  # Negative score for negative sentiment
        return {'finbert_score': scores}
    except Exception as e:
        print(f"Error processing batch. Error: {e}")
        return {'finbert_score': [0] * len(batch['summary'])}

# Use tqdm to show progress

print("Synonym Replacing: ")
dataset_synonymed = dataset_synonymed.map(get_finbert_scores_batch, batched=True, batch_size=8)
print("Entity Masking: ")
dataset_masked = dataset_masked.map(get_finbert_scores_batch, batched=True, batch_size=8)
print("Sentence Reordering: ")
dataset_reordered = dataset_reordered.map(get_finbert_scores_batch, batched=True, batch_size=8)
print("Synonym->Entity Mask->Sentence Reorder: ")
dataset_all_augmented = dataset_all_augmented.map(get_finbert_scores_batch, batched=True, batch_size=8)

#-------------------------
# Convert back to pandas DataFrame
df = dataset_synonymed.to_pandas()

# Display the DataFrame with the new column
print(df.head(10))

# Save the new CSV
df.to_csv("AUGMENTED_synonym_merged_news_stock(finbert).csv", index=False)
print("Saved as AUGMENTED_synonym_merged_news_stock(finbert).csv")
print("# Rows: ", str(len(df)))

#------------------------
# Convert back to pandas DataFrame
df = dataset_masked.to_pandas()

# Display the DataFrame with the new column
#print(df.head(10))

# Save the new CSV
df.to_csv("AUGMENTED_entityMask_merged_news_stock(finbert).csv", index=False)
print("Saved as AUGMENTED_entityMask_merged_news_stock(finbert).csv")
print("# Rows: ", str(len(df)))

#------------------------
# Convert back to pandas DataFrame
df = dataset_reordered.to_pandas()

# Display the DataFrame with the new column
#print(df.head(10))

# Save the new CSV
df.to_csv("AUGMENTED_reorder_merged_news_stock(finbert).csv", index=False)
print("Saved as AUGMENTED_reorder_merged_news_stock(finbert).csv")
print("# Rows: ", str(len(df)))

#------------------------
# Convert back to pandas DataFrame
df = dataset_all_augmented.to_pandas()

# Display the DataFrame with the new column
#print(df.head(10))

# Save the new CSV
df.to_csv("ALL_AUGMENTED_SynMaskReorder_merged_news_stock(finbert).csv", index=False)
print("Saved as ALL_AUGMENTED_SynMaskReorder_merged_news_stock(finbert).csv")
print("# Rows: ", str(len(df)))



*****Augmenting data with SYNONYMS*****
Displaying first 10 augmented examples: 
Original:  2024 was a good year for stocks, with the S&P 500 up 26.6% for the year. The economy grew more robustly than many expected while the Federal Reserve began cutting interest rates, making credit and lending less expensive and giving businesses and consumers some financial breathing room.
Augmented:  2024 was a good year for stock, with the S & P 500 upward 26. 6% for the year. The economy grew more robustly than many expected while the Fed Reserve began cutting interest rates, making credit and lending less expensive and give businesses and consumers some financial breathing elbow room. 

Original:  Verizon offers three-year price lock and free phone for new and existing myPlan customers, boosting value amid competitive market. Verizon's new plan includes free satellite text messaging, but price hikes on fees and taxes remain possible, experts warn. Feel unsure about the market's next move?
Augmen

Map:   0%|          | 0/21651 [00:00<?, ? examples/s]

Entity Masking: 


Map:   0%|          | 0/21600 [00:00<?, ? examples/s]

Sentence Reordering: 


Map:   0%|          | 0/16988 [00:00<?, ? examples/s]

Synonym->Entity Mask->Sentence Reorder: 


Map:   0%|          | 0/21652 [00:00<?, ? examples/s]

  ticker                                              title  \
0   AAPL  Mag 7 Earnings Preview: What Can Investors Exp...   
1   AAPL                  Should You Invest in Bitcoin Now?   
2   AAPL  Apple Plans iPhone Shift From China To India A...   
3   AAPL                               Earnings Data Deluge   
4   AAPL  Comparing Apple With Industry Competitors In T...   
5   AAPL  Apple, Google, And Tesla Help Sentiment, As Te...   
6   AAPL  S&P 500, Magnificent 7 Hit Resistance: Analyst...   
7   AAPL  Conflicting Claims on U.S./China Trade Sends P...   
8   AAPL  Technical Analysis For Trading SPY And Top Tec...   
9   AAPL             2 Top Stocks You Can Buy Now With $500   

                                             summary         sentiment  \
0  We get into the heart of the Q1 earnings seaso...           Neutral   
1  Bitcoin has broken away from the NASDAQ and is...  Somewhat-Bullish   
2  Apple plans to import most U.S. iPhones from I...           Neutral   
3  Pre-mar